In [153]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait as wait
import time
import pandas as pd

In [191]:
def get_jobs(keyword, num_jobs, verbose, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/shuoliu/Documents/GitHub/ds_salary_proj/chromedriver",options = options)
    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locT=&locId=&locKeyword=&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    #url = "https://www.glassdoor.com/Job/jobs.htm?sc.generalKeyword=data+scientist&sc.locationSeoString=san+francisco&locId=1147401&lo_IP2.htm?radius=100"
    #url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&locT=&locId=&jobType=&context=Jobs&sc.keyword=data+scientist&dropdown=0"
    #url = "https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KE0,14_IP2.htm"
    driver.get(url)
    time.sleep(10)
    jobs = []
    
    
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(5)
        
        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass
        
      
        #job_buttons = driver.find_elements_by_xpath(".//a[@class='jobInfoItem jobTitle css-13w0lq6 eigr9kq1 jobLink']")  #jl for Job Listing. These are the buttons we're going to click.
        job_buttons = driver.find_elements_by_class_name("react-job-listing")
        for job_button in job_buttons:  
            
            driver.implicitly_wait(5)
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            job_button.click()
            time.sleep(10)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    #company_name = driver.find_element_by_xpath('.//div[@class="jobInfoItem jobEmpolyerName]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    #location = driver.find_element_by_xpath('.//div[contains(@class,"empLoc"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    #job_title = driver.find_element_by_xpath('.//div[contains(@class, "jobTitle")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                    
                except:
                    time.sleep(slp_time)
                    
            try:
                application_link = driver.find_element_by_xpath('.//div[@class="fixedApply"]//a').get_attribute('href')
            except:
                application_link = -1
                
            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."
                
                
            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))
                print("Application Link:{}".format(application_link))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1
                    
                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors,
            "Application Link" : application_link            })
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
    return pd.DataFrame(jobs)



In [192]:
#This line will open a new chrome window and start the scraping.
df = get_jobs('data scientist', 10, False, 15)
df

Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Application Link
0,Data Scientist,$80K-$134K (Glassdoor est.),Company Description\n\nMMIT is growing from th...,3.9,"Managed Markets Insight & Technology, LLC\n3.9","Yardley, PA",-1,201 to 500 Employees,1994,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,-1,https://www.glassdoor.com/partner/jobListing.h...
1,Sr Data-Applied Scientist-PCCI,$80K-$134K (Glassdoor est.),"Looking for your next opportunity? If so, choo...",4.4,Parkland Center for Clinical Innovation (PCCI)...,"Dallas, TX",-1,1 to 50 Employees,2012,Nonprofit Organization,IT Services,Information Technology,Unknown / Non-Applicable,-1,https://www.glassdoor.com/partner/jobListing.h...
2,Medical Technologist (MT) / Medical Laboratory...,$80K-$134K (Glassdoor est.),Sampson Regional Medical Center is hiring a Me...,2.2,Sampson Regional Medical Center\n2.2,"Clinton, NC",-1,501 to 1000 Employees,1949,Hospital,Health Care Services & Hospitals,Health Care,$50 to $100 million (USD),-1,-1
3,Revit Expert / BIM Modeling Engineer - Concret...,$80K-$134K (Glassdoor est.),We are seeking a professional familiar with co...,-1,Palatial Concrete,"Alexandria, VA",-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Data Engineer,$80K-$134K (Glassdoor est.),The Data Engineer will work with product owner...,2.8,Chenega Corporation\n2.8,"Springfield, VA",-1,5001 to 10000 Employees,1974,Company - Private,Aerospace & Defense,Aerospace & Defense,$500 million to $1 billion (USD),-1,https://www.glassdoor.com/partner/jobListing.h...
5,Senior Data Scientist,$80K-$134K (Glassdoor est.),The Mars Agency is a global marketing practice...,2.9,The Mars Agency\n2.9,"Southfield, MI",-1,201 to 500 Employees,1972,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1,https://www.glassdoor.com/partner/jobListing.h...
6,Scientist,$80K-$134K (Glassdoor est.),"Join us as we Rise to the Challenge\nAt KCI, w...",3.6,KCI Technologies Inc.\n3.6,"Sparks, MD",-1,1001 to 5000 Employees,1988,Company - Private,Architectural & Engineering Services,Business Services,$100 to $500 million (USD),-1,https://www.glassdoor.com/partner/jobListing.h...
7,Scientist,$80K-$134K (Glassdoor est.),ArsenalBio’s mission is to develop efficacious...,5.0,ArsenalBio\n5.0,"South San Francisco, CA",-1,51 to 200 Employees,2019,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,-1,https://www.glassdoor.com/partner/jobListing.h...
8,Business Intelligence Analyst,$80K-$134K (Glassdoor est.),About Canal Insurance\n\nCanal Insurance Compa...,3.5,Canal Insurance Company\n3.5,"Greenville, SC",-1,201 to 500 Employees,1939,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD),-1,https://www.glassdoor.com/partner/jobListing.h...
9,"Senior Scientist, Machine Learning",$80K-$134K (Glassdoor est.),Do you want to be part of a growing data organ...,4.2,Niantic Inc\n4.2,"Sunnyvale, CA",-1,501 to 1000 Employees,2011,Company - Private,Video Games,Media,Unknown / Non-Applicable,-1,-1


In [193]:
df.to_csv('ds_salary_proj.csv',index = False)